In [1]:
%matplotlib inline
import pandas as pd


In [2]:
purchases = pd.read_csv("../data/treated/december_purchases_clean.csv")
purchases.dtypes

Unnamed: 0        int64
NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Expansion of data  that we already have

In [3]:
purchases["FECHA"] = pd.to_datetime(purchases["FECHA"])

In [4]:
# Splitting the date data into three different columns, day, month and year
date = purchases["FECHA"]
purchases = purchases.assign(DIA = date.dt.day, MES = date.dt.month, ANO = date.dt.year)

## Projecting variables agains other ones

In [5]:
# Creating a new column with the amount of purchases maade for each beneficiary
beneficiario_sales = purchases["BENEFICIARIO"].value_counts().to_dict()
purchases["BENEFICIARIO_SALES"] = purchases["BENEFICIARIO"].apply(lambda x: beneficiario_sales[x])

In [6]:
# Creating a variable with the mean of purchases separated by sheet
sheet_mean_purchases = purchases.groupby("SHEET")["IMPORTE"].mean()
sheet_mean_purchases

SHEET
COMUNICACION              109705.343662
CONTRATISTAS Y FDO FED    474566.765787
GASTOS VARIOS             881659.731797
SERV PPROF                229592.166667
SERV. PERS.                59623.405217
Name: IMPORTE, dtype: float64

## Take info from anoher sources 

In [11]:
pd.DataFrame(purchases["BENEFICIARIO"].unique()).to_csv("../data/treated/unique_beneficiarios.csv", index=None)

In [12]:
# Importing data scrapped with pahntom buster
linkedin_mty = pd.read_csv("../data/raw/linkedin_mty.csv")

In [13]:
# Creates a dictionary where indicates who has linkefIn and who doesn't
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele["query"]] = True if ele["error"] == True else False

In [15]:
# Add a column that indicate if the beneficiary has or not linkedin (replace("  ", " ") is to avoid error becouse double spacing)
purchases["HAS_LINKEDIN"] = purchases["BENEFICIARIO"].apply(lambda x: has_site[x.replace("  ", " ")])

In [16]:
purchases["HAS_LINKEDIN"].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN, dtype: int64

In [17]:
purchases.to_csv("../data/treated/purchases_enriched.csv", index=None)